In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_csv("/home/ricardo/Github/trump_taiwan_tweets_analysis/datasets/original_tweet_text_merged_tab.csv", sep = "\t")

# remove duplicates rows
print('Before:', len(data))
data = data.drop_duplicates()
print('After: ', len(data))

# sort df values
data.sort_values(by=['Tweet'])

# create new data frame by account
for i, j in enumerate(tqdm(data.iloc[:, 0])):
    tweet_split_list = j.split(" ")
    # check if the tweet is retweeted
    if "RT" in tweet_split_list:
        data.loc[i, 'if_RT'] = 1
        RT_index = tweet_split_list.index('RT')
        # original text
        original_text = ""
        for a, b in enumerate(tweet_split_list[:RT_index]):
            original_text += " " + b
        data.loc[i, "original_text"] = original_text
        # Retweeted Text
        retweeted_text = ""
        for a, b in enumerate(tweet_split_list[RT_index:]):
            retweeted_text += " " + b
        data.loc[i, "retweeted text"] = retweeted_text
    else:
        data.loc[i, 'if_RT'] = 0
        # original text
        original_text = ""
        for a, b in enumerate(tweet_split_list):
            original_text += " " + b
        data.loc[i, "original_text"] = original_text
        
        
data.to_csv("/home/ricardo/Desktop/temp.csv", sep='\t')

  0%|          | 40/31316 [00:00<01:18, 396.20it/s]

Before: 61989
After:  31316


100%|██████████| 31316/31316 [00:53<00:00, 590.67it/s]
